In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from libs.ds_charts import get_variable_types

# Missing values imputation

In [5]:
dataset_2 = pd.read_csv('dataset_2/air_quality_tabular.csv', na_values='NaN')
new_dataset_2 = dataset_2.copy()

#FIND VARIABLES WITH MISSING VALUES
mv = {}
for var in new_dataset_2:
    nr = new_dataset_2[var].isna().sum()
    if nr > 0:
        mv[var] = nr

#DISCARD COLUMNS WITH MORE THEN 90% MISSING VALUES
threshold = new_dataset_2.shape[0] * 0.90

missings = [c for c in mv.keys() if mv[c]>threshold]
new_dataset_2.drop(columns=missings, inplace=True)
print('Dropped variables', missings)

#DISCARD RECORDS WITH MAJORITY OF MISSING VALUES
threshold = new_dataset_2.shape[1] * 0.50

new_dataset_2.dropna(thresh=threshold, inplace=True)
print(new_dataset_2.shape)

for column in mv:
    if column != "Field_1":
        vars = dataset_2[column]
        mean_vars = int(vars.mean())
        dataset_2[column].fillna(mean_vars,inplace=True)
    # else: # VER FIELD_1

Dropped variables []
(161631, 32)


# Dummification

In [7]:
file = 'air_quality'
filename = 'data/air_quality.csv'
symbolic_vars = ['City_EN', 'Prov_EN']

def dummify(df, vars_to_dummify):
    other_vars = [c for c in df.columns if not c in vars_to_dummify]
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=bool)
    X = df[vars_to_dummify]
    encoder.fit(X)
    new_vars = encoder.get_feature_names(vars_to_dummify)
    trans_X = encoder.transform(X)
    dummy = pd.DataFrame(trans_X, columns=new_vars, index=X.index)
    dummy = dummy.convert_dtypes(convert_boolean=True)

    final_df = pd.concat([df[other_vars], dummy], axis=1)
    return final_df

variables = get_variable_types(new_dataset_2)
df = dummify(new_dataset_2, symbolic_vars)
df.to_csv(f'data/{file}_dummified.csv', index=False)

df.describe(include=[bool])

,City_EN_Abacangzuqiangzu,City_EN_Akesudiqu,City_EN_Alashan,City_EN_Aletaidiqu,City_EN_Alidiqu,City_EN_Ankang,City_EN_Anqing,City_EN_Anshan,City_EN_Anshun,City_EN_Anyang,...,Prov_EN_Shaanxi,Prov_EN_Shandong,Prov_EN_Shanghai,Prov_EN_Shanxi,Prov_EN_Sichuan,Prov_EN_Tianjin,Prov_EN_Xinjiang,Prov_EN_Xizang,Prov_EN_Yunnan,Prov_EN_Zhejiang
count,161631,161631,161631,161631,161631,161631,161631,161631,161631,161631,...,161631,161631,161631,161631,161631,161631,161631,161631,161631,161631
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,161265,161182,161182,161182,161182,161182,161182,161182,161182,161182,...,157141,149151,161182,156692,152451,161182,154878,158489,154450,155016
